In [1]:
import pandas as pd

# 加载数据
file_path = "data/data.csv"
df = pd.read_csv(file_path)

# 保留数据的 1-7 行（Python 中索引从 0 开始）
df_subset = df.iloc[:7]

# 查看数据结构
df_info = df_subset.info()
df_head = df_subset.head()


df_info, df_head


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Name   6 non-null      object 
 1   Country Code   6 non-null      object 
 2   Series Name    6 non-null      object 
 3   Series Code    6 non-null      object 
 4   1991 [YR1991]  6 non-null      float64
 5   1992 [YR1992]  6 non-null      float64
 6   1993 [YR1993]  6 non-null      float64
 7   1994 [YR1994]  6 non-null      float64
 8   1995 [YR1995]  6 non-null      float64
 9   1996 [YR1996]  6 non-null      float64
 10  1997 [YR1997]  6 non-null      float64
 11  1998 [YR1998]  6 non-null      float64
 12  1999 [YR1999]  6 non-null      float64
 13  2000 [YR2000]  6 non-null      float64
 14  2001 [YR2001]  6 non-null      float64
 15  2002 [YR2002]  6 non-null      float64
 16  2003 [YR2003]  6 non-null      float64
 17  2004 [YR2004]  6 non-null      float64
 18  2005 [YR2005] 

(None,
     Country Name Country Code  \
 0  United States          USA   
 1  United States          USA   
 2  United States          USA   
 3         Canada          CAN   
 4         Canada          CAN   
 
                                          Series Name        Series Code  \
 0  Ratio of female to male labor force participat...  SL.TLF.CACT.FM.ZS   
 1  Unemployment, total (% of total labor force) (...     SL.UEM.TOTL.ZS   
 2  Labor force participation rate, total (% of to...     SL.TLF.ACTI.ZS   
 3  Ratio of female to male labor force participat...  SL.TLF.CACT.FM.ZS   
 4  Unemployment, total (% of total labor force) (...     SL.UEM.TOTL.ZS   
 
    1991 [YR1991]  1992 [YR1992]  1993 [YR1993]  1994 [YR1994]  1995 [YR1995]  \
 0      75.189205      75.731187      76.309203      78.001061      78.402472   
 1       6.800000       7.500000       6.900000       6.120000       5.650000   
 2      74.712000      75.080000      75.052000      75.252000      75.438000   
 3   

In [2]:
# 转换为长表格式（Long Format）
df_long = pd.melt(df_subset, 
                  id_vars=['Country Name', 'Country Code', 'Series Name', 'Series Code'], 
                  var_name='Year', 
                  value_name='Value')

# 提取年份并转换为整数
df_long['Year'] = df_long['Year'].str.extract(r'(\d{4})').astype(int)

# 处理缺失值（可选）：删除缺失值行
df_long.dropna(inplace=True)

# 显示处理后的数据
df_long



,Country Name,Country Code,Series Name,Series Code,Year,Value
0,United States,USA,Ratio of female to male labor force participat...,SL.TLF.CACT.FM.ZS,1991,75.189205
1,United States,USA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,1991,6.800000
2,United States,USA,"Labor force participation rate, total (% of to...",SL.TLF.ACTI.ZS,1991,74.712000
3,Canada,CAN,Ratio of female to male labor force participat...,SL.TLF.CACT.FM.ZS,1991,77.865302
4,Canada,CAN,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,1991,10.320000
...,...,...,...,...,...,...
218,United States,USA,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,2022,3.650000
219,United States,USA,"Labor force participation rate, total (% of to...",SL.TLF.ACTI.ZS,2022,72.675000
220,Canada,CAN,Ratio of female to male labor force participat...,SL.TLF.CACT.FM.ZS,2022,88.141206
221,Canada,CAN,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,2022,5.280000


In [3]:
# 重新整理数据，使其符合 国家-年份-多个变量 的格式

# 透视数据，使每个 "Series Name" 作为单独的列
df_pivot = df_long.pivot_table(index=['Country Name', 'Country Code', 'Year'], 
                               columns='Series Name', 
                               values='Value').reset_index()

# 重置列名格式（去掉多层索引）
df_pivot.columns.name = None

# 显示处理后的数据
df_pivot

# 指定导出文件路径
export_path = "df_pivot.csv"

# 将 DataFrame 导出为 CSV 文件，不包含索引
df_pivot.to_csv(export_path, index=False)




In [4]:
# 确保 'Year' 列为整数类型
df_pivot['Year'] = df_pivot['Year'].astype(int)

# 按年份划分数据集
train_df = df_pivot[df_pivot['Year'] <= 2020]  # 1991-2020 作为训练集
test_df = df_pivot[df_pivot['Year'] > 2020]   # 2021-2022 作为测试集

train_df_USA = train_df = df_pivot[df_pivot['Country Code'] == 'USA']
train_df_CAN = train_df = df_pivot[df_pivot['Country Code'] == 'CAN']



## ADF檢驗
Augmented Dickey-Fuller Test（簡稱ADF檢驗）是一種統計檢驗方法，用於檢測時間序列數據是否具有單位根，即判斷該時間序列是否是平穩的（stationary）。平穩性是時間序列分析的重要特性，許多統計模型（如 ARIMA）要求數據是平穩的才能正確建模。

## 檢驗假設
- 原假設 (Null Hypothesis, H0)：數據具有單位根（Unit Root），即數據是非平穩的
- 備擇假設 (Alternative Hypothesis, H1)：數據不具有單位根，即數據是平穩的。
- 使用p value判斷是否接受或拒絕原假設：
    - p 值 > 5%：數據非平穩，需要進一步處理
    - p 值 ≤ 5%：數據平穩，可以用於 ARIMA 模型

In [5]:
from statsmodels.tsa.stattools import adfuller

# 对训练集中的所有数值列进行 ADF 检验
adf_results = {}

for column in train_df_USA.columns[3:]:  # 跳过前3列（Country Name, Country Code, Year）
    series = train_df[column].dropna()  # 移除 NaN 值以确保 ADF 检验可执行
    
    if len(series) > 1:  # 需要至少两个数据点进行ADF检验
        result = adfuller(series)
        adf_results[column] = {
            'ADF Statistic': result[0],
            'p-value': result[1],
            'Critical Values': result[4],
            'Stationary': result[1] <= 0.05  # 是否平稳
        }



# 转换结果为 DataFrame 方便查看
adf_df = pd.DataFrame.from_dict(adf_results, orient='index')
adf_df


,ADF Statistic,p-value,Critical Values,Stationary
"Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)",-0.653894,0.858333,"{'1%': -3.661428725118324, '5%': -2.9605253412...",False
Ratio of female to male labor force participation rate (%) (modeled ILO estimate),-3.324424,0.013821,"{'1%': -3.7238633119999998, '5%': -2.98648896,...",True
"Unemployment, total (% of total labor force) (modeled ILO estimate)",-2.489765,0.117985,"{'1%': -3.6699197407407405, '5%': -2.964070740...",False


## 檢驗結果
- 該數據非平穩，需要利用差分（Differencing） 將數據轉化為平穩數據

## 差分
- 在 Auto ARIMA 中，差分用參數d表示

- d=0：數據已平穩，無需差分。
- d=1：對數據執行一次差分。
- d=2：對數據執行兩次差分，依此類推。
- Auto ARIMA 會自動選擇適合的 d 值來處理非平穩性。

In [6]:
# 进行一阶差分，使数据平稳化
train_diff = train_df_USA.copy()

for column in train_df_USA.columns[3:]:  # 只对数值列进行差分
    train_diff[column] = train_df_USA[column].diff()

# 移除因差分导致的 NaN 值
train_diff.dropna(inplace=True)

# 重新进行 ADF 检验，检查差分后的平稳性
adf_diff_results = {}

for column in train_diff.columns[3:]:  
    series = train_diff[column].dropna()
    
    if len(series) > 1:  
        result = adfuller(series)
        adf_diff_results[column] = {
            'ADF Statistic': result[0],
            'p-value': result[1],
            'Critical Values': result[4],
            'Stationary': result[1] <= 0.05  # 是否平稳
        }

adf_diff_results_USA_df = pd.DataFrame.from_dict(adf_diff_results, orient='index')
# 显示差分后的 ADF 检验结果

adf_diff_results_USA_df



,ADF Statistic,p-value,Critical Values,Stationary
"Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)",-3.975763,0.001543,"{'1%': -3.6699197407407405, '5%': -2.964070740...",True
Ratio of female to male labor force participation rate (%) (modeled ILO estimate),-3.861210,0.002340,"{'1%': -3.6699197407407405, '5%': -2.964070740...",True
"Unemployment, total (% of total labor force) (modeled ILO estimate)",-4.894210,0.000036,"{'1%': -3.6699197407407405, '5%': -2.964070740...",True


In [7]:
from pmdarima import auto_arima

forecast_values = {}

for column in train_diff.columns[3:]:
    series = train_diff[column].dropna()
    
    if len(series) > 5:  # 确保有足够的数据点
        model = auto_arima(series, seasonal=False, stepwise=True)
        forecast = model.predict(n_periods=2)
        forecast_values[column] = forecast


forecast_values


c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the n

{'Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)': 31    0.169086
 32    0.047492
 dtype: float64,
 'Ratio of female to male labor force participation rate (%) (modeled ILO estimate)': 31    0.156422
 32    0.156422
 dtype: float64,
 'Unemployment, total (% of total labor force) (modeled ILO estimate)': 31    0.0
 32    0.0
 dtype: float64}

In [8]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pmdarima import auto_arima

# 训练 ARIMA 模型并预测
arima_results = {}
forecast_values = {}

for column in train_diff.columns[3:]:  # 只对数值列进行 ARIMA 训练和预测
    series = train_diff[column].dropna()  # 移除 NaN 确保建模

    if len(series) > 5:  # 确保数据足够进行建模
        # 使用 auto_arima 自动选择最佳 (p,d,q) 参数
        model = auto_arima(series, seasonal=False, stepwise=True, suppress_warnings=True)
        
        # 预测 2021 和 2022 年的数据
        forecast = model.predict(n_periods=2)
        forecast_values[column] = forecast

        # 存储模型结果
        arima_results[column] = model

# 转换预测结果为 DataFrame
future_years = [2021, 2022]
forecast_df = pd.DataFrame(forecast_values, index=future_years).reset_index()
forecast_df.rename(columns={'index': 'Year'}, inplace=True)

forecast_df


c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the n

,Year,"Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)",Ratio of female to male labor force participation rate (%) (modeled ILO estimate),"Unemployment, total (% of total labor force) (modeled ILO estimate)"
0,2021,NaN,NaN,NaN
1,2022,NaN,NaN,NaN
